In [465]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

In [466]:
import numpy as np
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [467]:
genesDf = pd.read_csv('C:/Users/Eli/Documents/2018_Fall/Data_Mining/Assignment2/Genes_relation.data')
genesDf.head(10)

,GeneID,Essential,Class,Complex,Phenotype,Motif,Chromosome,Function,Localization
0,G234064,Essential,GTP/GDP-exchange factors (GEFs),Translation complexes,?,PS00824,1,CELLULAR ORGANIZATION (proteins are localized ...,cytoplasm
1,G234064,Essential,GTP/GDP-exchange factors (GEFs),Translation complexes,?,PS00824,1,PROTEIN SYNTHESIS,cytoplasm
2,G234064,Essential,GTP/GDP-exchange factors (GEFs),Translation complexes,?,PS00825,1,CELLULAR ORGANIZATION (proteins are localized ...,cytoplasm
3,G234064,Essential,GTP/GDP-exchange factors (GEFs),Translation complexes,?,PS00825,1,PROTEIN SYNTHESIS,cytoplasm
4,G234065,Non-Essential,ATPases,?,?,?,1,"CELL RESCUE, DEFENSE, CELL DEATH AND AGEING",cytoplasm
5,G234065,Non-Essential,ATPases,?,?,?,1,CELLULAR ORGANIZATION (proteins are localized ...,cytoplasm
6,G234065,Non-Essential,ATPases,?,?,?,1,CELLULAR TRANSPORT AND TRANSPORTMECHANISMS,cytoplasm
7,G234065,Non-Essential,ATPases,?,?,?,1,PROTEIN DESTINATION,cytoplasm
8,G234065,Non-Essential,Molecular chaperones,?,?,?,1,"CELL RESCUE, DEFENSE, CELL DEATH AND AGEING",cytoplasm
9,G234065,Non-Essential,Molecular chaperones,?,?,?,1,CELLULAR ORGANIZATION (proteins are localized ...,cytoplasm


In [468]:
genesDf.shape

(4346, 9)

Replace ? values with null

In [469]:
genesDf = genesDf.replace('?', np.NaN)

Remove data we don't need, "Function"

In [470]:
genesDf = genesDf.drop(columns=['Function'])

In [471]:
list(genesDf)

['GeneID',
 'Essential',
 'Class',
 'Complex',
 'Phenotype',
 'Motif',
 'Chromosome',
 'Localization']

Display number of null values in the columns as a percentage

In [472]:
1 - genesDf.count()/len(genesDf.index)

GeneID          0.000000
Essential       0.030603
Class           0.611367
Complex         0.434883
Phenotype       0.245053
Motif           0.515186
Chromosome      0.000460
Localization    0.000000
dtype: float64

Class, Motif and Complex have a lot of missing data so we'll drop them

In [473]:
genesDf = genesDf.drop(columns=['Class','Motif','Complex'])

In [474]:
1 - genesDf.count()/len(genesDf.index)

GeneID          0.000000
Essential       0.030603
Phenotype       0.245053
Chromosome      0.000460
Localization    0.000000
dtype: float64

Now we'll try removing all the rows that have null values

In [475]:
genesDf = genesDf.dropna(how='any',axis=0) 

In [476]:
genesDf.shape

(3236, 5)

In [477]:
1 - genesDf.count()/len(genesDf.index)

GeneID          0.0
Essential       0.0
Phenotype       0.0
Chromosome      0.0
Localization    0.0
dtype: float64

There are still 3200+ rows left and all our null values have been removed
Now lets remove GeneID since it has no value to us

In [478]:
genesDf = genesDf.drop(columns=['GeneID'])

In [479]:
list(genesDf)

['Essential', 'Phenotype', 'Chromosome', 'Localization']

Next we'll seperate the data into train, test and predictors vs target

In [480]:
n =  2400
seed = 1
train = genesDf.sample(n,random_state = seed).sort_index()
test = genesDf[~genesDf.isin(train).all(1)].sort_index()

In [481]:
train.shape

(2400, 4)

In [482]:
test.shape

(836, 4)

Now let's train a decision tree on Localization data

In [483]:
ytrain = train[['Localization']] 
xtrain = train [['Essential','Phenotype','Chromosome']]
dtree = DecisionTreeClassifier()
xtrain['Essential'] = pd.factorize(xtrain['Essential'])[0]
xtrain['Phenotype'] = pd.factorize(xtrain['Phenotype'])[0]
dtree.fit(xtrain,ytrain)
pred = dtree.predict(xtrain)
dtree.score(xtrain,ytrain)

C:\Users\Eli\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Eli\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


0.6233333333333333

Our decision tree has a 62% accuracy.
Now lets import our test data

In [484]:
testDf = pd.read_csv('C:/Users/Eli/Documents/2018_Fall/Data_Mining/Assignment2/Genes_relation.test')

Now lets prepare the test data.

In [485]:
testDf = testDf.replace('?', np.NaN)
testDf = testDf.drop(columns=['Localization','Function','Class','Motif','Complex'])
testDf = testDf.dropna(how='any',axis=0) 
geneId = testDf[['GeneID']]
list(testDf)

['GeneID', 'Essential', 'Phenotype', 'Chromosome']

Now let's run the test test data through our decision tree

In [486]:
xtest = testDf [['Essential','Phenotype','Chromosome']]
xtest['Essential'] = pd.factorize(xtest['Essential'])[0]
xtest['Phenotype'] = pd.factorize(xtest['Phenotype'])[0]
pred = dtree.predict(xtest)
pred = pd.DataFrame(pred)
pred.columns = ['Localization']
pred

C:\Users\Eli\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Eli\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Localization
0,nucleus
1,plasma membrane
2,cytoplasm
3,cytoskeleton
4,nucleus
5,ER
6,cytoskeleton
7,cytoplasm
8,nucleus
9,plasma membrane


Now we'll join the gene ids back to the prediction dataframe so we can compare our predictions to the actual

In [487]:
geneId = geneId.reset_index()
pred = pd.DataFrame({'Localization_p' : pred.Localization, 'GeneID' : geneId.GeneID})
pred.head(10)

,Localization_p,GeneID
0,nucleus,G234273
1,plasma membrane,G234490
2,cytoplasm,G234394
3,cytoskeleton,G234385
4,nucleus,G234365
5,ER,G235818
6,cytoskeleton,G235319
7,cytoplasm,G236758
8,nucleus,G234247
9,plasma membrane,G237124


Now we'll read in the actual data

In [493]:
keys = pd.read_csv('C:/Users/Eli/Documents/2018_Fall/Data_Mining/Assignment2/keys.txt')
keys.columns = ['GeneID','Localization_a']
keys.head(10)

,GeneID,Localization_a
0,G234062,mitochondria
1,G234081,nucleus
2,G234086,mitochondria
3,G234087,nucleus
4,G234094,nucleus
5,G234097,nucleus
6,G234112,mitochondria
7,G234120,plasma membrane
8,G234125,golgi
9,G234137,cytoskeleton


And now we'll merge the actual data with the predicted data using the Gene ID

In [494]:
results = pred.merge(keys, on='GeneID')
results.head(10)

,Localization_p,GeneID,Localization_a
0,nucleus,G234273,cytoplasm
1,nucleus,G234273,cytoplasm
2,vacuole,G234273,cytoplasm
3,vacuole,G234273,cytoplasm
4,vacuole,G234273,cytoplasm
5,nucleus,G234273,cytoplasm
6,vacuole,G234273,cytoplasm
7,nucleus,G234273,cytoplasm
8,plasma membrane,G234490,nucleus
9,nucleus,G234490,nucleus


Now let's compare the predicted vs the actual to determine our accuracy

In [497]:
results['Correct'] = np.where(results['Localization_p'] == results['Localization_a'], 1, 0)
results.head(10)

,Localization_p,GeneID,Localization_a,Correct
0,nucleus,G234273,cytoplasm,0
1,nucleus,G234273,cytoplasm,0
2,vacuole,G234273,cytoplasm,0
3,vacuole,G234273,cytoplasm,0
4,vacuole,G234273,cytoplasm,0
5,nucleus,G234273,cytoplasm,0
6,vacuole,G234273,cytoplasm,0
7,nucleus,G234273,cytoplasm,0
8,plasma membrane,G234490,nucleus,0
9,nucleus,G234490,nucleus,1


In [507]:
results.Correct.sum()/results.shape[0]

0.2672672672672673

The predictor was right 26.7% of the time. Not very good, but it's a start.